In [1]:
import numpy as np, matplotlib, time, copy, random, math
%load_ext line_profiler
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

### Utils

In [2]:
def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = [int(n) for n in content[1].split()]#tuple([int(n) for n in content[1].split()])
    libraries = []
    def book_sort(book):
        return bookValues[book]
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = tuple(sorted([int(id) for id in content[2+2*i+1].split()], key = book_sort, reverse=True))
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [3]:
def check_solution(D, libraries):
    days = 0
    prev_books = set()
    for library in libraries:
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days < D)
    
def score_solution(libraries, book_values):
    score=0
    for library in libraries:
        for book_id in library.book_ids:
            score+= book_values[book_id]
    return score

### Classes

In [4]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
        
    def get_n_best_books(self,n, book_values):
        result = [];
        k=0
        for i in range(len(self.book_ids)):
            if book_values[self.book_ids[i]] != 0:
                result.append(self.book_ids[i])
                k+=1
                if k == n:
                    return tuple(result)
        return tuple(result)
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

In [5]:
class ProblemSolver:
    def __init__(self, B, L, D, book_values, libraries):
        self.B, self.L, self.D = B,L,D
        self.book_values = book_values
        self.libraries = libraries
        
    def get_solution(self,selected_lib_ids=None):
        if not selected_lib_ids:
            selected_lib_ids = self.get_individual()
        local_book_values = copy.copy(self.book_values)
        day = 0
        selected_libraries = [copy.copy(self.libraries[i]) for i in selected_lib_ids]
        it=0;
        while it < len(selected_libraries):
            next_library = selected_libraries[it]
            day+=next_library.signup_time
            if day>=self.D:
                break
            next_library.book_ids = next_library.get_n_best_books((self.D-day)*next_library.books_per_day, local_book_values)
            for book_id in next_library.book_ids:
                local_book_values[book_id] = 0
            it+=1
        return selected_libraries[:it]
    
    def get_individual(self):
        pass

In [6]:
class HeurysticSolver(ProblemSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time
    
    def get_individual(self):
        lib_ids = [i for i in range(len(self.libraries))]
        lib_ids.sort(key = self.library_score, reverse= True)
        day = 0
        for i in range(len(lib_ids)):
            day+=self.libraries[lib_ids[i]].signup_time
            if day >= self.D:
                return tuple(lib_ids[:i])
        return tuple(lib_ids)

In [7]:
class PowerSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time**(1+lib.signup_time/self.D)

In [8]:
class SimpleScoreVarianceSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores/book_variance

In [9]:
class SquareScoreVarianceSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores**2/(lib.signup_time*lib.signup_time*math.sqrt(book_variance))

In [10]:
class BookNumbersSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores**2/(book_variance*len(n_best_books)*lib.signup_time)

In [11]:
class ScoreSquareSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, self.book_values)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores**2/lib.signup_time

In [14]:
sum_score = 0
solvers_cls = [HeurysticSolver, PowerSolver, SimpleScoreVarianceSolver, SquareScoreVarianceSolver, BookNumbersSolver, ScoreSquareSolver]
for file_path in file_paths:
    (B,L,D),book_values, libraries = process_file(file_path)
    solvers = [cl(B,L,D,book_values, libraries) for cl in solvers_cls]
    solutions = [solver.get_solution() for solver in solvers]
    print(list(lib.id for lib in solutions[0]))
    [check_solution(D, solution) for solution in solutions]
    scores = [score_solution(solution, book_values) for solution in solutions]
    print(scores)
    score = max(scores)
    print(score)
    sum_score+=score
print("SUM",sum_score)

[0, 1]
[21, 21, 21, 21, 21, 21]
21
[38, 92, 27, 29, 48, 73, 77, 11, 34, 66, 88, 21, 32, 65, 75, 82, 18, 37, 57, 76, 2, 9, 22, 62, 14, 16, 31, 89, 13, 42, 46, 52, 53, 67, 69, 79, 84, 36, 40, 43, 55, 56, 61, 86, 90, 6, 30, 64, 78, 91, 59, 99, 7, 15, 58, 83, 3, 4, 12, 20, 28, 41, 45, 63, 81, 87, 5, 19, 23, 26, 85, 94, 44, 47, 50, 72, 74, 93, 8, 25, 33, 39, 70, 71, 80, 98, 54, 68, 0, 1]
[5822900, 5822900, 4075800, 5822900, 5822900, 5822900]
5822900
[1949, 8696, 4871, 6550, 4705, 9772, 8384, 2297, 7327, 2610, 5724, 3428, 8530, 6241, 5523, 8617, 8738, 1634, 8917, 3953, 6897, 1864, 4758, 8728, 53, 9829, 7989, 8691, 1045, 4193, 2359, 7281, 18, 8149, 9255, 1760, 627, 3938, 349, 8791, 7491, 9447, 5134, 1161, 2380, 9092, 177, 650, 9128, 1023, 111, 6871, 198, 5771, 8887, 6644, 7819, 2006, 2882, 5348, 8758, 8181, 7791, 2346, 661, 7358, 8490, 122, 8279, 7984, 4074, 6754, 5804, 806, 4460, 2732, 5020, 7709, 7305, 9927, 416, 9855, 5339, 1549, 479, 1653, 3679, 1349, 8005, 1283, 6063, 2754, 4059, 1448, 5

[600, 594, 522, 972, 602, 717, 100, 209, 178, 597, 808, 61, 715, 157, 393, 72, 466, 59, 690, 390, 708, 933, 813, 4, 37, 952, 495, 626, 478, 902, 854, 506, 607, 491, 87, 693, 736, 270, 476, 244, 387, 241, 343, 158, 27, 125, 23, 970, 959, 779, 665, 82, 350, 700, 669, 239, 232, 555, 148, 388, 69, 577, 556, 828, 852, 995, 691, 117, 894, 520, 678, 335, 589, 993, 848, 830, 969, 971, 304, 142, 639, 359, 630, 320, 512, 795, 793, 267, 150, 944, 60, 737, 455, 621, 633, 366, 268, 410, 587, 462, 411, 606, 443, 583, 834, 220, 205, 191, 245, 563, 210, 844, 0, 216, 112, 373, 566, 780, 962, 772, 815, 105, 314, 575, 447, 798, 988, 179, 794, 403, 288, 45, 581]
[4613373, 4642132, 4056632, 4606497, 4514009, 2040620]
4642132
[204, 210, 694, 719, 595, 774, 901, 618, 369, 422, 312, 622, 672, 828, 209, 454, 147]
[5240161, 5238624, 1195114, 5211764, 4111186, 5083231]
5240161
SUM 26166356


In [13]:
sum_score = 0
for file_path in file_paths:
    (B,L,D),book_values, libraries = process_file(file_path)
    solver = HeurysticSolver(B,L,D,book_values, libraries)
    solution = solver.get_solution()
    check_solution(D, solution)
    power_solver = PowerSolver(B,L,D,book_values, libraries)
    power_solution = power_solver.get_solution()
    check_solution(D, power_solution)
    score = max(score_solution(solution, book_values), score_solution(power_solution, book_values))
    print(score)
    sum_score +=score
print(sum_score)

21
5822900
5645747
4815395
4642132
5240161
26166356
